In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import shutil

In [3]:
from subprocess import Popen, PIPE
import time
from IPython.display import clear_output

In [4]:
PATH = "/home/gridsan/ptheron/MoEBERT-fork"
PYPATH = "/home/gridsan/ptheron/.conda/envs/MoEBERT/bin/python"
LOG_PATH = f"{PATH}/logs"

In [5]:
LOG_PATH

'/home/gridsan/ptheron/MoEBERT-fork/logs'

# Download glue datasets and necessary module from huggingface

In [6]:
!bash ../fetching/sh_dl_ds.sh

python: can't open file 'sh_scripts/fetching/dl_ds.py': [Errno 2] No such file or directory


# Submit Jobs

In [7]:
## make the bash file for the jobs: copy the template and replace $1 by the dataset name
os.makedirs(f"{PATH}/sh_scripts/supercloud/ds_spec_submission", exist_ok=True)
name = "submit_hash" # name of the template do not precise the .sh
OUTPUT_DIR = '"/home/gridsan/$(whoami)/MoEBERT-fork/results"'
for ds in datasets:
    with open(f"{PATH}/sh_scripts/supercloud/ds_spec_submission/{name}_{ds}.sh", "w") as f:
        with open(f"{PATH}/sh_scripts/supercloud/{name}.sh", "r") as f2:
            for line in f2:
                if "$1" in line:
                    line = line.replace("$1", ds)
                if "OUTPUT_TOFILL" in line:
                    line = line.replace("OUTPUT_TOFILL", OUTPUT_DIR)
                f.write(line)            

In [22]:
datasets = ['rte','mrpc', 'sst2']

In [23]:
# create hash scripts
## make the bash file for the jobs: copy the template and replace $1 by the dataset name
os.makedirs(f"{PATH}/sh_scripts/supercloud/ds_spec_submission/hash", exist_ok=True)
name = "submit_hash" # name of the template do not precise the .sh
OUTPUT_DIR = '"/home/gridsan/$(whoami)/MoEBERT-fork/results"'
for num in range(1, 6):
    for ds in datasets:
        print(f"{name}_{ds}_{num}.sh")
        with open(f"{PATH}/sh_scripts/supercloud/ds_spec_submission/hash/{name}_{ds}_{num}.sh", "w") as f:
            with open(f"{PATH}/sh_scripts/supercloud/{name}.sh", "r") as f2:
                for line in f2:
                    if "$1" in line:
                        line = line.replace("$1", ds)
                    if "$2" in line:
                        line = line.replace("$2", str(num))
                    if "OUTPUT_TOFILL" in line:
                        line = line.replace("OUTPUT_TOFILL", OUTPUT_DIR)
                    f.write(line)            

submit_hash_rte_1.sh
submit_hash_mrpc_1.sh
submit_hash_sst2_1.sh
submit_hash_rte_2.sh
submit_hash_mrpc_2.sh
submit_hash_sst2_2.sh
submit_hash_rte_3.sh
submit_hash_mrpc_3.sh
submit_hash_sst2_3.sh
submit_hash_rte_4.sh
submit_hash_mrpc_4.sh
submit_hash_sst2_4.sh
submit_hash_rte_5.sh
submit_hash_mrpc_5.sh
submit_hash_sst2_5.sh


In [18]:
exit_code = 1
name = "submit_hash" # name of the template do not precise the .sh
ds = ['rte','mrpc','sst2']
to_run = [1, 2, 3, 4, 5]

#quit()  # to avoid running the script
submitted = []
for i in to_run:
    for ds_name in ds:
        if i % 100 == 0:
            clear_output(wait=True)
        print(i)
        while True:
            script_path = f"{PATH}/sh_scripts/supercloud/ds_spec_submission/hash/{name}_{ds_name}_{i}.sh"
            process = Popen(["LLsub", script_path], stdout=PIPE)        
            (output, err) = process.communicate()
            exit_code = process.wait()
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
            if exit_code == 0:
                print(f"submitted hash routing! {ds_name} {i}")
                tmp_id = str(output)[-11:-3]
                print("job id:", tmp_id)
                submitted.append(tmp_id)
                break
            time.sleep(50000)
        

1
2023-01-23 18:20:13 b'Submitted batch job 21194990\n' None
submitted hash routing! rte 1
job id: 21194990
1
2023-01-23 18:20:13 b'Submitted batch job 21194991\n' None
submitted hash routing! mrpc 1
job id: 21194991
1
2023-01-23 18:20:14 b'Submitted batch job 21194992\n' None
submitted hash routing! sst2 1
job id: 21194992
2
2023-01-23 18:20:14 b'Submitted batch job 21194993\n' None
submitted hash routing! rte 2
job id: 21194993
2
2023-01-23 18:20:14 b'Submitted batch job 21194994\n' None
submitted hash routing! mrpc 2
job id: 21194994
2
2023-01-23 18:20:15 b'Submitted batch job 21194995\n' None
submitted hash routing! sst2 2
job id: 21194995
3
2023-01-23 18:20:15 b'Submitted batch job 21194996\n' None
submitted hash routing! rte 3
job id: 21194996
3
2023-01-23 18:20:15 b'Submitted batch job 21194997\n' None
submitted hash routing! mrpc 3
job id: 21194997
3
2023-01-23 18:20:16 b'Submitted batch job 21194998\n' None
submitted hash routing! sst2 3
job id: 21194998
4
2023-01-23 18:20:16 

In [19]:
print(submitted)

['21194990', '21194991', '21194992', '21194993', '21194994', '21194995', '21194996', '21194997', '21194998', '21194999', '21195000', '21195002', '21195005', '21195006', '21195007']


Cancel jobs

hash-routing

In [24]:
submitted = ['21195354', '21195355', '21195356', '21195357', '21195358', '21195359', '21195360', '21195361', '21195362', '21195363', '21195364', '21195365', '21195366', '21195367', '21195368']

In [25]:
for job in submitted:
    process = Popen(['scancel',job], stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code ==0:
        print(job, "deleted!")

21195246 deleted!
21195248 deleted!
21195249 deleted!
21195250 deleted!
21195251 deleted!
21195252 deleted!
21195253 deleted!
21195254 deleted!
21195255 deleted!
21195256 deleted!
21195257 deleted!
21195258 deleted!
21195259 deleted!
21195260 deleted!
21195261 deleted!


### BERT Task specific finetuning

In [9]:
torun = ['rte', 'cola', 'mrpc', 'sst2']
submitted = []

In [1]:
exit_code = 1
name = "submit_perm" # name of the template do not precise the .sh
for i,ds_name in enumerate(torun):
    if i % 100 == 0:
        clear_output(wait=True)
    print(i)
    while True:
        script_path = f"{PATH}/sh_scripts/supercloud/ds_spec_submission/{name}_{ds_name}.sh"
        process = Popen(["sbatch", script_path], stdout=PIPE)
        (output, err) = process.communicate()
        exit_code = process.wait()
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
        if exit_code == 0:
            print(ds_name,"submitted!")
            tmp_id = str(output)[-11:-3]
            print("job id:", tmp_id)
            submitted.append(tmp_id)
            break
        time.sleep(50000)
        

NameError: name 'torun' is not defined

### MoEBERT Seeds finetuning

In [94]:
torun = datasets.copy()

30


In [ ]:
submitted = []

In [95]:
exit_code = 1
name = "submit_seed" # name of the template do not precise the .sh
for i,all_id in enumerate(torun):
    if i % 100 == 0:
        clear_output(wait=True)
    print(i)
    while True:
        script_path = f"{PATH}/sh_scripts/supercloud/ds_spec_submission/{name}_{ds_name}.sh"
        process = Popen(["sbatch", script_path], stdout=PIPE)        
        (output, err) = process.communicate()
        exit_code = process.wait()
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
        if exit_code == 0:
            print(all_id,"submitted!")
            tmp_id = str(output)[-11:-3]
            print("job id:", tmp_id)
            submitted.append(tmp_id)
            break
        time.sleep(50000)
        

0
2023-01-06 11:27:09 b'Submitted batch job 21011683\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed61.sh submitted!
job id: 21011683
1
2023-01-06 11:27:09 b'Submitted batch job 21011684\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed62.sh submitted!
job id: 21011684
2
2023-01-06 11:27:09 b'Submitted batch job 21011685\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed63.sh submitted!
job id: 21011685
3
2023-01-06 11:27:09 b'Submitted batch job 21011686\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed64.sh submitted!
job id: 21011686
4
2023-01-06 11:27:09 b'Submitted batch job 21011687\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed65.sh submitted!
job id: 21011687
5
2023-01-06 11:27:09 b'Submitted batch job 210116

# Cancel jobs

In [625]:
for job in submitted:
    process = Popen(['scancel',job], stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code ==0:
        print(job, "deleted!")

20989143 deleted!
20989144 deleted!
20989145 deleted!
20989146 deleted!
20989147 deleted!
20989148 deleted!
20989149 deleted!
20989150 deleted!
20989151 deleted!
20989152 deleted!
20989153 deleted!
20989154 deleted!
20989155 deleted!
20989156 deleted!
20989157 deleted!
20989158 deleted!
20989159 deleted!
20989160 deleted!
20989161 deleted!
20989162 deleted!
20989163 deleted!
20989164 deleted!
20989165 deleted!
20989166 deleted!
20989167 deleted!
20989168 deleted!
20989169 deleted!
20989170 deleted!
20989171 deleted!
20989172 deleted!
20989173 deleted!
20989174 deleted!
20989175 deleted!
20989176 deleted!
20989177 deleted!
20989178 deleted!
20989179 deleted!
20989180 deleted!
20989181 deleted!
20989182 deleted!
20989183 deleted!
20989184 deleted!
20989185 deleted!
20989186 deleted!
20989187 deleted!
20989188 deleted!
20989189 deleted!
20989190 deleted!
20989191 deleted!
20989192 deleted!
20989193 deleted!
20989194 deleted!
20989195 deleted!
20989196 deleted!
20989197 deleted!
20989198 d